In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
from engine.models.distance import DistanceMatrix
from engine.models.movie import Movie
import pandas as pd

In [3]:
folder = '/media/mariam/Files/ran/clacket-save'
matrix = DistanceMatrix(folder)
matrix.load()

In [4]:
movies = ['1'] + list(matrix.matrix['1'].keys())

In [42]:
import sys

DATASET = 'dataset/expanded_movies.txt'
QUALIFYING = 'dataset/qualifying.txt'
folder = '/media/mariam/Files/ran/clacket-save'

def get_present_qualifying(movies):
    present = {}
    curr = None
    with open(QUALIFYING) as qualifying:
        for line in qualifying:
            if ':' in line: # movie id line
                id = line.split(':')[0]
                if id in movies:
                    curr = id
                    present[id] = []
                else:
                    curr = None
            elif curr is not None:
                user = line.split(',')[0]
                present[id].append(user)
    return present

def available_users(present, ks=[1,2,3,4]):
    available = {}
    i = 1
    n = len(present)
    for movie_id in present:
        sys.stdout.write('\rCounting available ratings for {0} out of {1}'.format(i, n))
        sys.stdout.flush()
        movie = Movie.load(folder, movie_id)
        qualifying_users = present[movie_id]
        available[movie_id] = {}
        for k in ks:
            users = movie.predictions[9399][k]['users']
            pres = len([u for u in users if str(u) in qualifying_users])
            absent = len(qualifying_users) - pres
            available[movie_id][k] = (pres, absent)
        i += 1
    return available
            

In [25]:
present = get_present_qualifying(movies)

In [43]:
available = available_users(present)

Counting available ratings for 9271 out of 9271

In [44]:
available

{'1': {1: (0, 53), 2: (0, 53), 3: (0, 53), 4: (1, 52)},
 '1000': {1: (0, 29), 2: (0, 29), 3: (0, 29), 4: (0, 29)},
 '10000': {1: (0, 3), 2: (0, 3), 3: (0, 3), 4: (0, 3)},
 '10003': {1: (0, 4), 2: (0, 4), 3: (1, 3), 4: (1, 3)},
 '10006': {1: (3, 6), 2: (3, 6), 3: (3, 6), 4: (3, 6)},
 '10008': {1: (0, 8), 2: (0, 8), 3: (0, 8), 4: (0, 8)},
 '1001': {1: (16, 221), 2: (11, 226), 3: (11, 226), 4: (31, 206)},
 '10010': {1: (0, 40), 2: (0, 40), 3: (0, 40), 4: (0, 40)},
 '10011': {1: (0, 13), 2: (0, 13), 3: (0, 13), 4: (0, 13)},
 '10012': {1: (0, 32), 2: (0, 32), 3: (2, 30), 4: (2, 30)},
 '10013': {1: (14, 85), 2: (10, 89), 3: (11, 88), 4: (29, 70)},
 '10014': {1: (0, 9), 2: (0, 9), 3: (0, 9), 4: (0, 9)},
 '10016': {1: (0, 12), 2: (0, 12), 3: (0, 12), 4: (0, 12)},
 '10018': {1: (0, 64), 2: (0, 64), 3: (1, 63), 4: (2, 62)},
 '10019': {1: (1, 131), 2: (0, 132), 3: (1, 131), 4: (1, 131)},
 '1002': {1: (0, 17), 2: (1, 16), 3: (1, 16), 4: (1, 16)},
 '10020': {1: (1, 64), 2: (0, 65), 3: (0, 65), 4: (

In [45]:
pr_t, ab_t = 0,0
for movie in available:
    for k in available[movie]:
        pr, ab = available[movie][k]
        pr_t += pr
        ab_t += ab
print('Present: {0}, Absent: {1}, Percentage: {2}%'.format(pr_t, ab_t, (pr_t / (ab_t + pr_t)) * 100))

Present: 180287, Absent: 8595621, Percentage: 2.054340132098012%


In [12]:
def rename_df(df):
    names = {}
    for min in [3,4,5]:
        names['perc_hits_{0}'.format(min)] = 'perc_hits_{0}'.format(min)
        names['mean_absolute_error_{0}'.format(min)] = 'MAE_{0}'.format(min)
        names['total_{0}'.format(min)] = 'total_{0}'.format(min)
        df['total_{0}'.format(min)] = df['total_hits_{0}'.format(min)] + df['total_misses_{0}'.format(min)]
        if min == 5:
            names['rmse_{0}'.format(min)] = 'RMSE'.format(min)
    return df.rename(columns=names)[['k'] + list(names.values())].round(3)

def filt(df, min):
    return df[['k', 'MAE_{0}'.format(min), 'perc_hits_{0}'.format(min), 'RMSE_{0}'.format(min)]]

In [59]:
df = rename_df(pd.read_pickle('2002_2005.pkl'))

In [60]:
df

,k,perc_hits_3,MAE_3,perc_hits_4,MAE_4,perc_hits_5,MAE_5,RMSE
0,1,85.181,0.865,63.043,0.835,37.500,1.344,1.648
1,2,81.486,0.979,56.659,1.049,33.223,1.188,1.591
2,3,80.346,1.000,56.931,1.066,34.049,1.185,1.574
3,4,88.187,0.775,73.197,0.727,51.574,0.715,1.351


In [26]:
dfmean = rename_df(pd.read_pickle('80_20.pkl'))
dfmin = rename_df(pd.read_pickle('80_20_min.pkl'))
dfmax = rename_df(pd.read_pickle('80_20_max.pkl'))

In [27]:
for k in [1,2,3,4]:
    for t in ['MAE', 'perc_hits']:
        dfm = dfmean[dfmean['k'] == k]
        dfmn = dfmin[dfmin['k'] == k]
        dfmx = dfmax[dfmax['k'] == k]
        dfm['type'] = 'mean'
        dfmn['type'] = 'min'
        dfmx['type'] = 'max'
        res = pd.concat([dfm, dfmx, dfmn])[['type', '{0}_3'.format(t), '{0}_4'.format(t), '{0}_5'.format(t)]]
        res.reset_index(inplace=True, drop=True)
        res.to_pickle('80_20_min_max_k_{0}_{1}.pkl'.format(k, t))

/home/mariam/code/clacket/clacket/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/mariam/code/clacket/clacket/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/mariam/code/clacket/clacket/venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [32]:
df = pd.read_pickle('80_20_min_max_k_{0}_{1}.pkl'.format(3, 'MAE'))

In [33]:
df

,type,MAE_3,MAE_4,MAE_5
0,mean,0.547,0.546,0.632
1,max,0.782,0.827,1.036
2,min,0.724,0.732,0.906


In [121]:
res

,type,MAE_3,MAE_4,MAE_5
0,mean,0.979,1.049,1.188
1,max,0.979,1.049,1.188
2,min,0.979,1.049,1.188


In [122]:
res.to_pickle('2002_2005_min_max.pkl')

In [13]:
df = rename_df(pd.read_pickle('80_20.pkl'))

In [14]:
df

,k,perc_hits_3,MAE_3,total_3,perc_hits_4,MAE_4,total_4,perc_hits_5,MAE_5,total_5,RMSE
0,1,96.560,0.251,1662685,89.930,0.243,1076372,83.253,0.272,428516,0.701
1,2,93.707,0.446,2521990,81.045,0.443,1612724,69.248,0.505,606940,0.918
2,3,92.398,0.547,3344812,76.843,0.546,2125256,61.369,0.632,773082,1.006
3,4,91.731,0.603,4027770,75.332,0.591,2490883,58.621,0.677,876365,1.057
